In [19]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime

# config
API_KEY = '619a280fbcf44de6be0170103250109'
CITY = 'London' , 'Paris'

# extract
def extract_weather_data(api_key, city):
    url = f'http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no'
    response = requests.get(url)
    response.raise_for_status() # raise error if bad response
    return response.json()

# transform
def transform_weather_data(raw_json):
    current = raw_json['current']
    location = raw_json['location']
    data = {
        'city': location['name'],
        'country': location['country'],
        'temperature': current['temp_c'],
        'humidity': current['humidity'],
        'wind_speed': current['wind_kph'],
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    return pd.DataFrame([data])

# Load
def load_to_sqlite(df, db_name = 'weather_data.db', table_name = 'weather'):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()

# pipeline
if __name__ == '__main__':
    for city in CITY:
      raw_json = extract_weather_data(API_KEY, city)
      df = transform_weather_data(raw_json)
      load_to_sqlite(df)
      print ('Data loaded into SQLite database')

   # export data to csv
    conn = sqlite3.connect('weather_data.db')
    df = pd.read_sql_query('SELECT * FROM weather', conn)
    df.to_csv('weather_data.csv', index=False)
    print ('Data exported to weather_data.csv')
    conn.close()



Data loaded into SQLite database
Data loaded into SQLite database
Data exported to weather_data.csv


In [17]:
!ls /content/

sample_data  weather_data.csv  weather_data.db	weather-etl
